In [86]:
import numpy 
import pandas 
import nltk
import csv
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

##### Los topicos se encuentran juntos en el dataframe, esta función los separa de acuerdo al tema, recibe el dataframe y devuelve una lista con los titulares de un solo tópico

In [87]:
def separaTopico(topico, df): #Separa todos los encabezados de acuerdo a su tópico
    listTopic = []
    dataArr = numpy.asarray(df)
    for i in range(len(dataArr)):
        if(dataArr[i][1] == topico):
            listTopic.append(dataArr[i][0])
    return listTopic

##### Cuando se tienen las palabras unicas de cada tópico, se eliminan caracteres especiales. Retorna una lista con las palabras que componen el texto

In [88]:
def limpiaTexto(text):
    "List all the word tokens in a text."
    return re.findall('[a-zA-Z0-9\-áéíóúÁÉÍÓÚ]{3,254}', text.lower())

##### Esta función busca eliminar palabras plurales para quedarnos únicamente con las singulares a través de la raíz de la palabra 

In [89]:
def RaizSingular(palabra):
    ultima = len(palabra) -1
    raiz = []
    plural = False

    if (palabra[ultima] == 's'): #Verificamos si la última letra es 's'
        if (palabra[ultima - 1] == 'a' or 
            palabra[ultima - 1] == 'i' or
            palabra[ultima - 1] == 'o' or
            palabra[ultima - 1] == 'u'):
            
            for i in range(ultima):
                    raiz.append(palabra[i])
            
            palabra = "".join(raiz)
            return palabra
            
        if(palabra[ultima - 1] == 'e'):
            if (palabra[ultima - 2] == 'c'):
                for i in range(ultima - 2):
                    raiz.append(palabra[i])
                    
                raiz.append('z')
                    
                palabra = "".join(raiz)
                return palabra
            
            elif (palabra[ultima - 2] != 'a' or
                palabra[ultima - 2] != 'e' or
                palabra[ultima - 2] != 'i' or
                palabra[ultima - 2] != 'o' or
                palabra[ultima - 2] != 'u'):
                
                for i in range(ultima - 1):
                    raiz.append(palabra[i])
                    
                palabra = "".join(raiz)
                return palabra
                                                                        
    else:
        return palabra

##### Esta función sustituye las tildes de las vocales por unicamente las vocales

In [90]:
def cleanTildes(palabra):
    s = palabra
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
    )
    
    for a, b in replacements:
        s = s.replace(a, b).replace(a.upper(), b.upper())
    
    return s

In [91]:
def limpiaPalabra(arrPalabras):
    raiz = []
    for i in range(len(arrPalabras)):
        #raiz.append(RaizSingular(cleanTildes(arrPalabras[i])))
        raiz.append(cleanTildes(arrPalabras[i]))
    
    return numpy.asarray(raiz, dtype=str)


##### Cada encabezado corresponde a un documento, esta función separa cada palabra para agregarlas a un solo array

In [92]:
def PalabrasTopico(listTopico): #Devuelve todas las palabras que aparecen en los titulares de las noticias 
    listHeadLines = []
    for i in range(len(listTopico)):
        #aux = nltk.word_tokenize(arrayTopico[i])
        aux = limpiaTexto(listTopico[i])
        #aux = limpiaPalabra()
        for j in range(len(aux)):
            aux2 = aux[j]
            listHeadLines.append(aux2)
            
    return numpy.asarray(listHeadLines)

In [93]:
def Probabilidad(palabrasTopico, palabrasUnicasTopico): #Devuelve un array con la ocurrencias de cada única del tópico
    Prob = numpy.zeros(len(palabrasUnicasTopico))
    for i in range(len(palabrasUnicasTopico)):
        Prob[i] = numpy.sum(palabrasTopico == palabrasUnicasTopico[i])
        Prob[i] = Prob[i] /len(palabrasTopico)
        
    return Prob

##### Recibe el vector de probabilidades y elige la máxima para finalmente devolver el tópico al que pertenece

In [94]:
def Clasificador(arrayProbabilidades):
    indice = numpy.where(arrayProbabilidades == max(arrayProbabilidades))[0][0]
    #print("Max = ", max(arrayProbabilidades))
    if (indice == 0):
        return 'covid'
    if (indice == 1):
        return 'tecnologia'
    if (indice == 2):
        return 'deportes'
    if (indice == 3):
        return 'economia'
    if (indice == 4):
        return 'cultura'
    return 'otros'

##### Devuelve la probabilidad máxima

In [109]:
def MAP(tweet, palabrasCovid, AlfabetoCovid, palabrasTecn, AlfabetoTecn, palabrasDep, AlfabetoDep, palabrasEcon, AlfabetoEcon, palabrasCult, AlfabetoCult):
    T = nltk.word_tokenize(tweet)
    Proba = numpy.ones(5)

    #Covid
    for j in range(len(T)):
        aux = numpy.sum(palabrasCovid == T[j]) + 1
        Proba[0] = (aux) / (len(AlfabetoCovid) + aux)
    
    #Tecnologia
    for j in range(len(T)):
        aux = numpy.sum(palabrasTecn == T[j]) 
        Proba[1] = (aux) / (len(AlfabetoTecn) + aux)

    #Deportes
    for j in range(len(T)):
        aux = numpy.sum(palabrasDep == T[j]) + 1
        Proba[2] = (aux) / (len(AlfabetoDep) + aux)
        
    #Economía
    for j in range(len(T)):
        aux = numpy.sum(palabrasEcon == T[j]) + 1
        Proba[3] = (aux) / (len(AlfabetoEcon) + aux)
        
    #Cultura
    for j in range(len(T)):
        aux = numpy.sum(palabrasCult == T[j]) + 1
        Proba[4] = (aux) / (len(AlfabetoCult) + aux)

    for p in range(len(Proba)):
        Proba[p] = Proba[p] * 1/5
        
    return Clasificador(Proba)
        

In [96]:
def limpiaTweet(df):
    listTweets = df.Tweets.values.tolist()
    pattern = re.compile('https?://(?:[-\w./]|(?:%\da-fA-F{2}))+')
    cleanTweets =[]
    
    for tweet in listTweets:
        res = re.findall(pattern, tweet)
        if (len(res) != 0):
            aux = tweet.replace(res[0], '')
            cleanTweets.append(aux)
        else:
            cleanTweets.append(tweet)
            
    for tweet in cleanTweets:
        if(len(tweet) == 0):
            cleanTweets.remove(tweet)
    
    return numpy.asarray(cleanTweets)

In [97]:
def preProcesamiento(listaTitulares):
    titulares = []
    for titular in listaTitulares:
        aux = limpiaTexto(titular)
        titulares.append(str.join(' ', aux))
    
    return titulares


##### TEST

#### Abrimos el dataset con pandas
Se abre el dataset de titulares y se realiza un preprocesamiento el cual consiste en eliminar tildes y eliminar palabras que tengan una longitud menor a tres 

In [98]:
nombres = ['Titular', 'Tópico']
dataTrain = pandas.read_csv('.//TitularesTrain//titulares.csv', names=nombres)
dataTrain = dataTrain.dropna()
dataTrain

,Titular,Tópico
0,Cierran rastro de Mérida tras detectar 17 cont...,covid
2,El “peor escenario” para Tabasco eran 200 muer...,covid
4,Jalisco instalará espacios para enfermos no gr...,covid
6,Guerrero no está listo para iniciar la “nueva ...,covid
8,Reportan la muerte de 364 personas por covid-1...,covid
...,...,...
7846,"“El Cascanueces” de Disney: Cuento de hadas, m...",cultura
7848,"Rock, blues y música alternativa en Sinaloa",cultura
7850,María Mercedes Coroy es “Malinche” en Canal Once,cultura
7852,“Motivos para el canto y la danza. Poesía del 68”,cultura


In [120]:
Y = dataTrain.Tópico.values.tolist()
X = dataTrain.Titular.values.tolist()

x, X_test, y, y_test = train_test_split(X, Y, test_size=0.3, random_state=100)

unir = []
for i in range(len(x)):
    unir.append([x[i], y[i]])
    


In [121]:
def splitTopic(topico, lista):
    tema = []
    for i in range(len(lista)):
        if (lista[i][1] == topico):
            tema.append(lista[i][0])
    
    return tema

##### Separamos del dataset por tópicos y a cada encabezado lo separamos por palabra de los documentos eliminando caracteres especiales

In [122]:
palabrasCovid = limpiaPalabra(PalabrasTopico(splitTopic('covid', unir)))
AlfabetoCovid = numpy.unique(palabrasCovid)
print("Alfabeto Covid = ", len(AlfabetoCovid))

palabrasTecn = limpiaPalabra(PalabrasTopico(splitTopic('tecnologia', unir)))
AlfabetoTecn = numpy.unique(palabrasTecn)
print("Alfabeto Tecnología = ", len(AlfabetoTecn))

palabrasDep = limpiaPalabra(PalabrasTopico(splitTopic('deportes', unir)))
AlfabetoDep = numpy.unique(palabrasDep)
print("Alfabeto Deportes = ", len(AlfabetoDep))

palabrasEcon = limpiaPalabra(PalabrasTopico(splitTopic('economia', unir)))
AlfabetoEcon = numpy.unique(palabrasEcon)
print("Alfabeto Economía = ", len(AlfabetoEcon))

palabrasCult = limpiaPalabra(PalabrasTopico(splitTopic('cultura', unir)))
AlfabetoCult = numpy.unique(palabrasCult)
print("Alfabeto Cultura = ", len(AlfabetoCult))

Alfabeto = numpy.concatenate((AlfabetoCovid, AlfabetoTecn, AlfabetoDep, AlfabetoEcon, AlfabetoCult))
Alfabeto.shape

Alfabeto Covid =  2261
Alfabeto Tecnología =  534
Alfabeto Deportes =  2446
Alfabeto Economía =  1988
Alfabeto Cultura =  1248


(8477,)

##### Obtenemos las probabilidades a priori, haciendo cada una de estas equiprobabloe $ P(C) $

In [123]:
PCovid = 1/5
PTecn = 1/5
PDep = 1/5
PEcon = 1/5
PCult = 1/5
print("PCovid = ", PCovid, "PTecn = ", PTecn, "PDep = ", PDep, "PEcon = ", PEcon, "PCult = ", PCult)

PCovid =  0.2 PTecn =  0.2 PDep =  0.2 PEcon =  0.2 PCult =  0.2


In [124]:
result = []
for i in range(len(X_test)):
    result.append(MAP(X_test[i], palabrasCovid, AlfabetoCovid, palabrasTecn, AlfabetoTecn, palabrasDep, AlfabetoDep, palabrasEcon, AlfabetoEcon, palabrasCult, AlfabetoCult))
    
resultClass = numpy.asarray(result)

print(len(resultClass))
print(len(y_test))

confusion_matrix(y_test, resultClass)

1178
1178


array([[134, 186,   4,  19,   3],
       [  5, 138,   2,   3,   0],
       [ 17, 248,  35,  21,   1],
       [  5, 212,   5,  90,   4],
       [  5,  33,   1,   5,   2]], dtype=int64)

### Test2

In [125]:
dataTweets = pandas.read_csv(".//TweetsTest//out.txt", names=['Tweets'])
dataTweets = dataTweets.dropna()
dataTweets.info()
dataTweets

<class 'pandas.core.frame.DataFrame'>
Int64Index: 678 entries, 0 to 677
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tweets  678 non-null    object
dtypes: object(1)
memory usage: 10.6+ KB


,Tweets
0,Tweets
1,https://t.co/6cmSHgMXJo
2,RT @AkemiLook: UPDATE: I just got off a call w...
3,RT @AkemiLook: CALLING LA LAWYERS: LA is in de...
4,"Pablo Guerrero Cañez, luchador social de Mexic..."
...,...
673,Baby Face https://t.co/Vp2AILGyTA
674,⟚ Ղ⚶▸ ̋ ᢇា឵឴┬┴┬┴┤(･_├┬┴┬┴ ͭ ͅ ⟘≷\n\ny̆̆̆̆̆̆̆̆...
675,https://t.co/ss4PjrxFGB
676,https://t.co/eb6dt2R3QD


In [126]:
tweets = limpiaTweet(dataTweets)
print(len(tweets))

622


In [127]:
Covid = []
Tecnologia = []
Deportes = []
Economia =[]
Cultura = []

for tweet in tweets:
    result = MAP(tweet, palabrasCovid, AlfabetoCovid, palabrasTecn, AlfabetoTecn, palabrasDep, AlfabetoDep, palabrasEcon, AlfabetoEcon, palabrasCult, AlfabetoCult)
    if result == 'covid':
        Covid.append([tweet, result])
    if result == 'tecnologia':
        Tecnologia.append([tweet, result])
    if result == 'deportes':
        Deportes.append([tweet, result])
    if result == 'economia':
        Economia.append([tweet, result])
    if result == 'cultura':
        Cultura.append([tweet, result])


In [128]:
print("Covid = ", len(Covid))
print("Tecnología = ", len(Tecnologia))
print("Deportes = ", len(Deportes))
print("Economía = ", len(Economia))
print("Cultura = ", len(Cultura))

Covid =  29
Tecnología =  3
Deportes =  2
Economía =  0
Cultura =  588
